## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [4]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [5]:
# 讀取資料集
wine = datasets.load_wine()
# 切分訓練集/測試集
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.25, random_state=42)
# 建立模型
cf_forest = RandomForestClassifier(n_estimators=20)

In [6]:
# 先看看使用預設參數得到的結果
cf_forest.fit(X_train, y_train)
y_pred = cf_forest.predict(X_test)
print("Acuuracy: ", metrics.accuracy_score(y_test, y_pred))

Acuuracy:  0.9777777777777777


In [7]:
# 設定要訓練的超參數組合
n_estimators = [20, 50, 100, 150, 200]
max_depth = [1, 3, 5, 7, 9]
criterion = ['gini', 'entropy']
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth, criterion=criterion)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(cf_forest, param_grid, scoring="accuracy", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 243 out of 250 | elapsed:    9.6s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   10.0s finished


In [8]:
# 印出最佳結果與最佳參數
print("Best Accuracy: {:.4f} using {}".format(grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.9852 using {'criterion': 'entropy', 'max_depth': 7, 'n_estimators': 20}


In [9]:
# 使用最佳參數重新建立模型
cf_forest_bestparam = RandomForestClassifier(n_estimators=grid_result.best_params_['n_estimators'],
                                             max_depth=grid_result.best_params_['max_depth'],
                                             criterion=grid_result.best_params_['criterion'],
                                             )

# 訓練模型
cf_forest_bestparam.fit(X_train, y_train)

# 預測測試集
y_pred = cf_forest_bestparam.predict(X_test)

In [ ]:
print("Acuuracy: ", metrics.accuracy_score(y_test, y_pred))